# CLLM generation & curation tutorial

# Setup keys and experiment

In [5]:
import sys
sys.path.append('src/')

from cllm.utils import *
from cllm.llm_gen import *
from cllm.data_loader import *

import pandas as pd
from copy import deepcopy
import time
from sklearn.model_selection import train_test_split


#############################################################
# API KEY SETUP INSTRUCTIONS
#############################################################

# for vllm
# api_key = "EMPTY"
# api_base = "http://localhost:8000/v1"

# for together
# api_key = "add together api key"
# api_base = "https://api.together.xyz/v1"


# for azure openai
# api_key = "EMPTY"
# api_base = "add azure deployment link"

# for openai
# api_key = "EMPTY"
# api_base = DO NOT INCLUDE

#############################################################

api_details = {
     "api_base": "add api base",
     "api_version": "2023-07-01-preview",
     "api_key": "add api key",
}


model_short_name = 'mixtral' # 'gpt-4' (do not use other short names)
model = "mistralai/Mixtral-8x7B-Instruct-v0.1" # "gpt4_20230815" (use name of your model deployment)
llm_serving='together' # supported 'azure_openai', 'together', 'vllm'

seed = 0
ns = 20 # n_samples per class. e.g. if binary = 40 samples (i.e. 20 per class)
dataset = 'compas'
n_synthetic=10 # just to test --- normall should be 1000
n_processes = 5

# STEP 1: Generation

## Get dataset

In [ ]:
df_feat, df_label, df = get_data(dataset=dataset, seed=seed)

X_train, X_remain, y_train, y_remain = sample_and_split(df_feat, df_label, ns=ns, seed=seed)

X_val, X_test, y_val, y_test = train_test_split(
    X_remain, y_remain, test_size=0.5, random_state=seed
)


X_train_orig = deepcopy(X_train)
y_train_orig = deepcopy(y_train)

## Setup Prompt

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


response_schemas = []

example_df = pd.concat([X_train_orig, y_train_orig], axis=1)

# Shuffle
example_df = example_df.sample(frac=1).reset_index(drop=True)


for idx, col in enumerate(list(example_df.columns)):
    if col == 'y':
        resp = ResponseSchema(name='y',
                        description=f"binary label, {col}", )
    else:
        resp = ResponseSchema(name=col,
                        description=f"feature column", )
    response_schemas.append(resp)

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()



generator_template = """\
You are a synthetic data generator. 
Your goal is to produce data which mirrors \
the given examples in causal structure and feature and label distributions \
but also produces as diverse samples as possible

I will give you real examples first

Leverage your knowledge about criminal recividsm to generate 1000 realistic but diverse samples. 

example data: {data}

{format_instructions}

DO NOT COPY THE EXAMPLES but generate realistic but new and diverse samples which have the correct label conditioned on the features.
"""


prompt = ChatPromptTemplate.from_template(template=generator_template)

## Generate using LLM

In [ ]:
retries = 4  # Max retries you want to attempt

while retries > 0:
    try:

        if len(example_df)>20:
            ic_samples=20
        else:
            ic_samples=len(example_df)
        
        print(f'Running {dataset}, {seed}, {model} --- {n_processes}')
        df_llm = llm_gen(prompt, generator_template, format_instructions, example_df, 
                        n_samples=n_synthetic,
                        temperature=0.9,
                        max_tokens=1000, model=model, 
                        n_processes=n_processes,
                        ic_samples=ic_samples, 
                        llm_serving=llm_serving, 
                        api_details=api_details)
    
        
        break  # if successful, break out of the loop
    except Exception as e:
        time.sleep(120)
        print(f"Error: {e}. Retrying with reduced n_processes...")
        n_processes = int(n_processes/2)
        retries -= 1
        if n_processes < 1:
            print("Error: Minimum n_processes reached. Exiting...")
            break
# try:
tmp_df = df_llm.astype(example_df.dtypes)
df_llm = tmp_df
# except:
#     pass

In [ ]:
df_llm

## Process LLM generated data to have the same data types

In [ ]:
df_llm = df_llm.dropna()
df_llm = df_llm[~df_llm.apply(lambda row: any([isinstance(cell, str) and cell in ['integer', 'float', 'numeric', 'categorical', 'number', 'No', 'Yes', 'continuous', 'age in years', 'string'] for cell in row]), axis=1)]

example_df = deepcopy(X_train_orig)
example_df['y'] = deepcopy(y_train_orig)

try:
    df_llm = df_llm.astype(example_df.dtypes)
except:
    # Assuming the dtypes from the example_df['Dtrain'].dataframe() is what you want
    target_dtypes = example_df.dtypes.to_dict()

    problematic_rows = set()

    for col, dtype in target_dtypes.items():
        for index, value in df[col].items():
            try:
                _ = dtype.type(value)  # Try to convert the value
            except Exception:
                problematic_rows.add(index)

    # Convert the problematic rows to a list and sort them
    problematic_rows = sorted(list(problematic_rows))

    # Drop the problematic rows
    df_llm.drop(problematic_rows, inplace=True)

    # Identify rows where any cell is of type list
    rows_with_lists = df.applymap(lambda x: isinstance(x, list)).any(axis=1)

    # Drop those rows
    df_llm = df_llm[~rows_with_lists]

    df_llm = df_llm.astype(example_df.dtypes)


df_llm

In [ ]:
df_llm.dtypes

# STEP 2: Curation

In [ ]:
from src.curation import data_centric_curation

X_check = df_llm.drop(columns=['y'])
y_check = df_llm['y'].values.astype(int)

curation_metric = 'aleatoric'
curation_ythresh=0.2
curation_xthresh=0 #adaptive

easy_train, ambig_train, unlearnable_train, Curator_xgb = data_centric_curation(X_train_orig, y_train_orig, X_check, y_check, 
                 curation_metric=curation_metric, retrain=False, nest = 100, 
                 curation_ythresh=curation_ythresh, curation_xthresh=curation_xthresh)

curated_train_ids = np.concatenate((easy_train, ambig_train))
curated_train_ids, unlearnable_train